In [ ]:
import itertools

from pytorch_toolbelt.utils.torch_utils import to_numpy
from sklearn.base import ClassifierMixin
from sklearn.decomposition import PCA
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from typing import Tuple

from retinopathy.models.heads import regression_to_class
from retinopathy.rounder import OptimizedRounder

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import pandas as pd
import numpy as np

In [ ]:
def _drop_features(df: pd.DataFrame):
    if 'features' in df:
        df = df.drop(columns=['features'])
    return df


def concat_folds_predictions(predictions, drop_features=False):
    dfs = []
    for fname in predictions:
        df = pd.read_pickle(fname)

        result = evaluate_predictions(df)
        print(fname, result)

        if drop_features:
            df = _drop_features(df)

        dfs.append(df)
    return pd.concat(dfs)


def evaluate_predictions(df: pd.DataFrame):
    if 'cls_diagnosis' not in df:
        cls_diagnosis = df['logits'].apply(np.argmax)
    else:
        cls_diagnosis = df['cls_diagnosis']

    if 'reg_diagnosis' not in df:
        reg_diagnosis = df['regression'].apply(regression_to_class).apply(int)
    else:
        reg_diagnosis = df['reg_diagnosis']

    if 'ord_diagnosis' not in df:
        ord_diagnosis = df['ordinal'].apply(regression_to_class).apply(int)
    else:
        ord_diagnosis = df['ord_diagnosis']

    y_true = df['diagnosis'].values
    cls_score = cohen_kappa_score(y_true,
                                  cls_diagnosis.values,
                                  weights='quadratic')

    reg_score = cohen_kappa_score(y_true,
                                  reg_diagnosis.values,
                                  weights='quadratic')

    ord_score = cohen_kappa_score(y_true,
                                  ord_diagnosis.values,
                                  weights='quadratic')

    # optR = OptimizedRounder()
    # optR.fit(df['regression'], df['diagnosis'])
    # coefficients = optR.coefficients()
    # reg_optimized = optR.predict(df['regression'], coefficients)
    #
    # reg_score_opt = cohen_kappa_score(df['diagnosis'].values,
    #                                         reg_optimized,
    #                                         weights='quadratic')

    # optR.fit(df['regression'], df['diagnosis'])
    # coefficients = optR.coefficients()
    # org_optimized = optR.predict(df['ordinal'], coefficients)
    #
    # ord_score_opt = cohen_kappa_score(df['diagnosis'].values,
    #                                         org_optimized,
    #                                         weights='quadratic')

    return {
        'cls_score': cls_score,
        'reg_score': reg_score,
        'ord_score': ord_score,
        # 'reg_score_opt': reg_score_opt,
        # 'ord_score_opt': ord_score_opt
    }


def prepare_oof_dataset(models_folds, use_features=False, use_predictions=True):
    x = []
    y = None

    for model_i, per_forld_predictions, in enumerate(models_folds):
        df = concat_folds_predictions(per_forld_predictions, drop_features=not use_features)
        y = df['diagnosis'].values

        if use_predictions:
            # logits = np.array(df['logits'].values.tolist())
            # x.append(logits)

            ordinal = np.array(df['ordinal'].tolist()).reshape(-1, 1)
            x.append(ordinal)

        if use_features:
            features = np.array(df['features'].values.tolist())
            x.append(features)

    x = np.concatenate(x, axis=1)
    # print(x.shape, y.shape)
    return x, y


def prepare_inference_datasets(models, use_features=False, use_predictions=True) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    x = []
    y = None
    y_average = []

    for model_i, predictions, in enumerate(models):
        df = pd.read_pickle(predictions)
        y = df['diagnosis'].values

        if use_predictions:
            # logits = np.array(df['logits'].values.tolist())
            # x.append(logits)

            ordinal = np.array(df['ordinal'].tolist()).reshape(-1, 1)
            x.append(ordinal)
            y_average.append(ordinal)

        if use_features:
            features = np.array(df['features'].values.tolist())
            x.append(features)

    x = np.concatenate(x, axis=1)
    y_average = np.concatenate(y_average, axis=1)
    y_average = np.mean(y_average, axis=1)
    y_average = to_numpy(regression_to_class(y_average))

    return x, y, y_average


def evaluate_on_datasets(predictor: ClassifierMixin, datasets):
    y_preds = []
    mean_kappa = []
    for i, (x, y_true) in enumerate(datasets):
        y_pred = predictor.predict(x)
        y_preds.append(y_pred)

        kappa_hold = cohen_kappa_score(y_true, y_pred, weights='quadratic')
        mean_kappa.append(kappa_hold)

    print(np.mean(mean_kappa), mean_kappa)
    return y_preds

In [ ]:
aptos2015_predictions = [
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold0_modest_williams_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold1_modest_williams_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold2_modest_williams_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold3_modest_williams_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold0_heuristic_sinoussi_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold1_heuristic_sinoussi_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold2_heuristic_sinoussi_aptos2015_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold3_heuristic_sinoussi_aptos2015_test_predictions.pkl',
]

aptos2019_predictions = [
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold0_modest_williams_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold1_modest_williams_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold2_modest_williams_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold3_modest_williams_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold0_heuristic_sinoussi_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold1_heuristic_sinoussi_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold2_heuristic_sinoussi_aptos2019_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold3_heuristic_sinoussi_aptos2019_test_predictions.pkl',

]

idrid_predictions = [
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold0_modest_williams_idrid_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold1_modest_williams_idrid_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold2_modest_williams_idrid_test_predictions.pkl',
    '../pretrained/Aug23_12_37_modest_williams/seresnext101_gap_512_medium_aptos2019_messidor_idrid_fold3_modest_williams_idrid_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold0_heuristic_sinoussi_idrid_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold1_heuristic_sinoussi_idrid_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold2_heuristic_sinoussi_idrid_test_predictions.pkl',
    '../pretrained/Aug23_heuristic_sinoussi/seresnext50_gap_512_medium_aptos2019_messidor_idrid_fold3_heuristic_sinoussi_idrid_test_predictions.pkl',
]

In [ ]:
use_features = True
random_state = 42
jobs = -1

In [ ]:
# use_features = True
# {'max_depth': 16, 'max_features': 32, 'n_estimators': 128}
# Optimized kappa 0.9208594025315285

# x, y = prepare_oof_dataset(oof_predictions, use_features=use_features, use_predictions=True)
# print(x.shape, y.shape)

holdout_x, holdout_y, holdout_y_avg = prepare_inference_datasets(idrid_predictions, use_features=use_features, use_predictions=True)
print('Holdout', holdout_x.shape, holdout_y.shape)
print('Holdout base score', cohen_kappa_score(holdout_y, holdout_y_avg, weights='quadratic'))

x, y, y_avg = prepare_inference_datasets(aptos2015_predictions, use_features=use_features, use_predictions=True)
print('Train', x.shape, y.shape)
print('Train base score', cohen_kappa_score(y, y_avg, weights='quadratic'))

if use_features:
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    holdout_x = scaler.transform(holdout_x)

    pca = PCA(n_components=512, random_state=42)
    print('Computing PCA')
    x = pca.fit_transform(x)
    holdout_x = pca.transform(holdout_x)
    print(x.shape)
else:
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    holdout_x = scaler.transform(holdout_x, y)